In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def extract_product_info(item):
    product = {}
    
    # 상품명
    title = item.find('div', class_='tit')
    product['title'] = title.text.strip() if title else 'N/A'
    
    # 상품 설명
    description = item.find('div', class_='cont')
    product['description'] = description.text.strip() if description else 'N/A'
    
    # # 여행 기간
    # period = item.find('dl', {'dt': '여행기간'})
    # product['duration'] = period.find('dd').text.strip() if period else 'N/A'
    
    # # 출발 기간
    # departure = item.find('dl', {'dt': '출발기간'})
    # product['departure_period'] = departure.find('dd').text.strip() if departure else 'N/A'

    
    # 여행 기간 및 요일 추출
    period = item.find('dl', {'dt': '여행기간'})
    if period:
        duration = period.find('dd')
        if duration:
            duration_text = duration.text.strip()
            product['duration'] = re.search(r'(\d+일)', duration_text).group(1) if re.search(r'(\d+일)', duration_text) else 'N/A'
            product['days'] = re.findall(r'[월화수목금토일]', duration_text)
        else:
            product['duration'] = 'N/A'
            product['days'] = []
    else:
        product['duration'] = 'N/A'
        product['days'] = []
    
    # 출발 기간 추출
    departure = item.find('dl', {'dt': '출발기간'})
    if departure:
        departure_period = departure.find('dd')
        product['departure_period'] = departure_period.text.strip() if departure_period else 'N/A'
    else:
        product['departure_period'] = 'N/A'
        
    
    # 항공사
    airline = item.find('ul', class_='flight')
    product['airline'] = airline.find('li').text.strip() if airline else 'N/A'
    
    # 상품 번호
    product_no = item.find('div', class_='no')
    product['product_no'] = product_no.text.strip().split()[-1] if product_no else 'N/A'
    
    # 가격
    price = item.find('div', class_='price')
    product['price'] = price.find('strong').text.strip() if price else 'N/A'
    
    # 라벨 정보
    labels = item.find_all('span', class_='label-state')
    product['labels'] = ', '.join([label.text.strip() for label in labels])
    
    # 자세히 보기 링크
    detail_button = item.find('button', class_='btn-type-1')
    if detail_button:
        product['detail_link'] = f"https://www.kyowontour.com/display/masterDetail?menuCode=M010105&productCode={detail_button.get('data-code')}"
    else:
        product['detail_link'] = 'N/A'
    
    
    return product


# 사용 예시
url = "https://www.kyowontour.com/display/masterMain?menuCode=M010105"
response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')
container = soup.select_one('body > div.wrap > main > div.cont-body > div > div > div.display-list-row')

# 각 상품 정보 추출
products = []
if container:
    items = container.find_all('div', class_='list-inner')
    for item in items:
        product_info = extract_product_info(item)
        products.append(product_info)

# 데이터프레임 생성
df = pd.DataFrame(products)
df


In [ ]:
df.loc[0]["detail_link"]